<table>
<tbody>
<tr>
<td width="80">&nbsp;</td>
<td width="20">&nbsp;</td>
</tr>
<tr>
<td>
<p style="color: #004571; font-size: 26px; width: 80%; background-color: white;"><strong>WHERE IS THE BETTER PLACE TO OPEN A RESTAURANT IN TORONTO CITY?</strong></p>
<p style="color: #004571; font-size: 26px; width: 80%; background-color: white;"><span style="color: #E57200;"><strong>Javier Cholbi Doblado</strong></span></p>
</td>
<td><img style="background-color: white;" title="Naturgy" src="https://upload.wikimedia.org/wikipedia/en/7/7d/Toronto_Coat_of_Arms.jpg" alt="Alt text that describes the graphic" width="256" height="256" align="right" /></td>
</tr>
</tbody>
</table>

## Problem to solve:

* One of our best clients wants to open a restaurant in the toronto city. He has a los of experience in the restaurants bussines (he has 5 restaurants in differents locations around the USA) but doesnt has information about the Toronto city...


* He would like to know the better place to put his restaurant, he hires our services to advice him about the best location with one condition, we must to focus on analyse deeply the available data and make a rigorous analysis using Machine Learning techniques.



## Description Data:
### For this analysis we must to follow the next steps:

* Find the available data about post codes and Neighbourhoods in Toronto.

* Find the available cardinals points for this Boroughs.

* Clean and transform the data using Data Wrangling techiques.

* Use Foursquare API to obtain information about restaurants in the differents Neighbourhoods.

* Draw in a map the different places using the Folium package from Python

* Use Machine Learning to divide the Map using the clustering techniques.

* Examine and analyse the clusters to obtain the best place to open the restaurant.


## Import the python package

In [3]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
from urllib.request import urlopen, Request
import folium
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import json
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [4]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
request = Request(url)
response = urlopen(request)
html = response.read()
df=pd.read_html(html, header=0)
df=df[0]
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


* Get the data in html format from wikipedia URL
* Put the html dataset into a dataframe

In [5]:
df=df.loc[df["Borough"]!='Not assigned']

In [6]:
df=df.replace('Not assigned',"Queen's Park")

* Filter the 'Not assigned' category from the initial dataframe
* Replace the Queen's Park problem

In [7]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [8]:
df.shape

(211, 3)

In [10]:
data = {'Postal Code': [],'Borough': [],'Neighbourhood': []}

for i in range(0,len(df)):
    
    data['Postal Code'].append(df.iloc[i,0])
    data['Borough'].append(df.iloc[i,1])
    data['Neighbourhood'].append(df.iloc[i,2])
           
df= pd.DataFrame(data)                       

* New dataframe to have the elements indexed from 0 to 210

In [11]:
df=df.groupby(['Postal Code', 'Borough'], as_index=False,sort=False).agg(','.join)

* Group by postcode and Borough and join with a commas...

In [12]:
df.head(10)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [13]:
df.shape

(103, 3)

In [14]:
path ='C:/Users/00022881/Downloads/'
name ='Geospatial_Coordinates.csv'
cor = pd.read_csv(path + name)
cor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


* Load the csv file with the Postal Cose Data

In [15]:
df_2=pd.merge(df, cor, on='Postal Code', how='inner')

* Merge both datasets using the merge function and inner mode

In [16]:
df_2.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


In [17]:
df_2.shape

(103, 5)

In [18]:
df_t=df_2.dropna()
df_t.columns= ['Postal Code','Borough','Neighborhood','Latitude','Longitude']
df_t.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


* Filter the dataset, only boroughts that contains the word Toronto

In [19]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

## Map Boroughts that contains the word Toronto

In [20]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df_t['Latitude'], df_t['Longitude'], df_t['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Define Foursquare Credentials and Version

In [82]:
CLIENT_ID = '5FDOWZUKJEDCINSPPKZOHKAOXUMV35A42TBYF5ETEHVDO33Z' # your Foursquare ID
CLIENT_SECRET = 'ZS42R0J0HYNNNYPRVP1DFMOFCX3EEBSAM2WK0G1CKZS2JJ25' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [84]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [85]:
toronto_venues = getNearbyVenues(names=df_t['Neighborhood'],
                                   latitudes=df_t['Latitude'],
                                   longitudes=df_t['Longitude']
                                  )



Parkwoods
Victoria Village
Harbourfront,Regent Park
Lawrence Heights,Lawrence Manor
Queen's Park
Islington Avenue
Rouge,Malvern
Don Mills North
Woodbine Gardens,Parkview Hill
Ryerson,Garden District
Glencairn
Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
Highland Creek,Rouge Hill,Port Union
Flemingdon Park,Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe
Guildwood,Morningside,West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor,Downsview North,Wilson Heights
Thorncliffe Park
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Scarborough Village
Fairview,Henry Farm,Oriole
Northwood Park,York University
East Toronto
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
East Birchmount Park,Ionview,Kennedy Park
Bayview Village
CFB Toronto,Downsview East
The Danforth West,Riverdale
Design E

## Let's check the size of the resulting dataframe

In [86]:
print(toronto_venues.shape)
toronto_venues.head()

(2241, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


## Let's check how many venues were returned for each neighborhood

In [87]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",10,10,10,10,10,10
"Alderwood,Long Branch",9,9,9,9,9,9
"Bathurst Manor,Downsview North,Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",23,23,23,23,23,23
Berczy Park,57,57,57,57,57,57


In [88]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 279 uniques categories.


## Analyze Each Neighborhood

In [89]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
del toronto_onehot['Neighborhood']


# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
toronto_onehot.shape

(2241, 279)

## Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [91]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head(10)

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.030000,...,0.010000,0.0,0.000000,0.0,0.0,0.01,0.0,0.0,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0
4,"Alderwood,Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0
5,"Bathurst Manor,Downsview North,Wilson Heights",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.055556,0.0,0.0,0.00,0.0,0.0,0.0,0.0
6,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0
7,"Bedford Park,Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,...,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0
8,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.017544,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0
9,"Birch Cliff,Cliffside West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0


In [92]:
toronto_grouped.shape

(99, 279)

In [93]:
num_top_venues = 3

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2  Thai Restaurant  0.04


----Agincourt----
            venue  freq
0  Clothing Store  0.25
1  Breakfast Spot  0.25
2          Lounge  0.25


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
               venue  freq
0         Playground   0.5
1               Park   0.5
2  Accessories Store   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
           venue  freq
0  Grocery Store   0.2
1    Pizza Place   0.2
2       Pharmacy   0.1


----Alderwood,Long Branch----
          venue  freq
0   Pizza Place  0.22
1  Skating Rink  0.11
2   Coffee Shop  0.11


----Bathurst Manor,Downsview North,Wilson Heights----
           venue  freq
0    Coffee Shop  0.11
1  Deli / Bodega  0.06
2     Restaurant  0.06


----Bayview Village----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  

2              Baseball Field  0.33


----Humber Summit----
                       venue  freq
0                Pizza Place   0.5
1        Empanada Restaurant   0.5
2  Middle Eastern Restaurant   0.0


----Humewood-Cedarvale----
          venue  freq
0         Field  0.33
1  Hockey Arena  0.33
2         Trail  0.33


----Kingsview Village,Martin Grove Gardens,Richview Gardens,St. Phillips----
         venue  freq
0  Pizza Place  0.25
1         Park  0.25
2     Bus Line  0.25


----Kingsway Park South West,Mimico NW,The Queensway West,Royal York South West,South of Bloor----
            venue  freq
0  Hardware Store  0.08
1   Grocery Store  0.08
2  Discount Store  0.08


----L'Amoreaux West----
                  venue  freq
0  Fast Food Restaurant  0.15
1    Chinese Restaurant  0.15
2           Coffee Shop  0.08


----Lawrence Heights,Lawrence Manor----
                    venue  freq
0  Furniture / Home Store  0.17
1          Clothing Store  0.17
2       Accessories Store  0.08


----L

## Let's put that into a *pandas* dataframe

In [94]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Now let's create the new dataframe and display the top 3 venues for each neighborhood.

In [156]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,Bar,Breakfast Spot,Cosmetics Shop,Restaurant,Burger Joint,American Restaurant
1,Agincourt,Lounge,Breakfast Spot,Clothing Store,Skating Rink,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Yoga Studio,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pizza Place,Fried Chicken Joint,Fast Food Restaurant,Sandwich Place,Coffee Shop,Beer Store,Pharmacy,Electronics Store,Eastern European Restaurant
4,"Alderwood,Long Branch",Pizza Place,Coffee Shop,Skating Rink,Sandwich Place,Pub,Dance Studio,Gym,Pharmacy,Comfort Food Restaurant,Electronics Store
5,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Deli / Bodega,Fast Food Restaurant,Restaurant,Sushi Restaurant,Supermarket,Diner,Fried Chicken Joint,Frozen Yogurt Shop,Sandwich Place
6,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
7,"Bedford Park,Lawrence Manor East",Italian Restaurant,Pizza Place,Coffee Shop,Fast Food Restaurant,Butcher,Thai Restaurant,Sandwich Place,Juice Bar,Restaurant,Pharmacy
8,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Steakhouse,Café,Cheese Shop,Seafood Restaurant,Farmers Market,Beer Bar,Bistro
9,"Birch Cliff,Cliffside West",College Stadium,General Entertainment,Café,Skating Rink,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dim Sum Restaurant


In [154]:
neighborhoods_venues_sorted.shape

(99, 4)

## Cluster Neighborhoods


In [157]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(init = "k-means++",n_clusters=kclusters, n_init = 12).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:200] 

array([0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 3, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 3, 0, 0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0,
       0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 3])

* Run *k*-means to cluster the neighborhood into 4 clusters.

## Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [135]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_t

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'),on='Neighborhood')
toronto_merged=toronto_merged.dropna()
toronto_merged.head(50) # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Fast Food Restaurant,Food & Drink Shop,Park,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Portuguese Restaurant,Coffee Shop,Intersection,Hockey Arena,Financial or Legal Service,French Restaurant,Pizza Place,Electronics Store,Eastern European Restaurant,Empanada Restaurant
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,2.0,Coffee Shop,Park,Bakery,Café,Pub,Gym / Fitness Center,Breakfast Spot,Mexican Restaurant,Beer Store,Bank
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,2.0,Furniture / Home Store,Clothing Store,Accessories Store,Boutique,Fraternity House,Event Space,Miscellaneous Shop,Coffee Shop,Women's Store,Vietnamese Restaurant
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,2.0,Coffee Shop,Gym,Park,Hobby Shop,Chinese Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burger Joint,Burrito Place
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,2.0,Fast Food Restaurant,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Field
7,M3B,North York,Don Mills North,43.745906,-79.352188,2.0,Gym / Fitness Center,Caribbean Restaurant,Baseball Field,Japanese Restaurant,Café,Convenience Store,Dive Bar,Falafel Restaurant,Event Space,Ethiopian Restaurant
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937,2.0,Fast Food Restaurant,Pizza Place,Intersection,Gym / Fitness Center,Pharmacy,Café,Athletics & Sports,Bank,Gastropub,Doner Restaurant
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,2.0,Coffee Shop,Clothing Store,Cosmetics Shop,Fast Food Restaurant,Middle Eastern Restaurant,Café,Ramen Restaurant,Theater,Plaza,Ice Cream Shop
10,M6B,North York,Glencairn,43.709577,-79.445073,0.0,Park,Pub,Pizza Place,Japanese Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop


In [137]:
toronto_merged_1=toronto_merged[toronto_merged["Cluster Labels"]==0]
toronto_merged_2=toronto_merged[toronto_merged["Cluster Labels"]==1]
toronto_merged_3=toronto_merged[toronto_merged["Cluster Labels"]==2]
toronto_merged_4=toronto_merged[toronto_merged["Cluster Labels"]==3]

In [284]:
toronto_merged_3.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,2.0,Coffee Shop,Park,Bakery,Café,Pub,Gym / Fitness Center,Breakfast Spot,Mexican Restaurant,Beer Store,Bank
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,2.0,Furniture / Home Store,Clothing Store,Accessories Store,Boutique,Fraternity House,Event Space,Miscellaneous Shop,Coffee Shop,Women's Store,Vietnamese Restaurant
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,2.0,Coffee Shop,Gym,Park,Hobby Shop,Chinese Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burger Joint,Burrito Place
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,2.0,Fast Food Restaurant,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Field
7,M3B,North York,Don Mills North,43.745906,-79.352188,2.0,Gym / Fitness Center,Caribbean Restaurant,Baseball Field,Japanese Restaurant,Café,Convenience Store,Dive Bar,Falafel Restaurant,Event Space,Ethiopian Restaurant


## Finally, let's visualize the resulting clusters

In [303]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_1['Latitude'], toronto_merged_1['Longitude'], toronto_merged_1['Neighborhood'], toronto_merged_1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7).add_to(map_clusters)
    
    
for lat, lon, poi, cluster in zip(toronto_merged_2['Latitude'], toronto_merged_2['Longitude'], toronto_merged_2['Neighborhood'], toronto_merged_2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_clusters)
       
        
for lat, lon, poi, cluster in zip(toronto_merged_3['Latitude'], toronto_merged_3['Longitude'], toronto_merged_3['Neighborhood'], toronto_merged_3['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_clusters)
    
    
for lat, lon, poi, cluster in zip(toronto_merged_4['Latitude'], toronto_merged_4['Longitude'], toronto_merged_4['Neighborhood'], toronto_merged_4['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.7).add_to(map_clusters)    
    
       
    
        
map_clusters

## Examine Clusters

# Requirments to put the shop in a Neighborhood:

Don't appear inside the 3st most common venues any restaurant or derivatives...we must to find a place without competency...

### Cluster 1:

In [234]:
cluester1=toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluester1.head()

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0.0,Portuguese Restaurant,Coffee Shop,Intersection,Hockey Arena,Financial or Legal Service,French Restaurant,Pizza Place,Electronics Store,Eastern European Restaurant,Empanada Restaurant
10,North York,0.0,Park,Pub,Pizza Place,Japanese Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
17,Etobicoke,0.0,Café,Pharmacy,Pizza Place,Liquor Store,Coffee Shop,Convenience Store,Beer Store,Yoga Studio,Drugstore,Dive Bar
22,Scarborough,0.0,Coffee Shop,Korean Restaurant,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
35,East York,0.0,Convenience Store,Park,Pizza Place,Coffee Shop,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop


In [288]:
cluester1=toronto_merged_1.iloc[:,1:9]
cluester1.head(12)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
1,North York,Victoria Village,43.725882,-79.315572,0.0,Portuguese Restaurant,Coffee Shop,Intersection
10,North York,Glencairn,43.709577,-79.445073,0.0,Park,Pub,Pizza Place
17,Etobicoke,"Bloordale Gardens,Eringate,Markland Wood,Old B...",43.643515,-79.577201,0.0,Café,Pharmacy,Pizza Place
22,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Dumpling Restaurant
35,East York,East Toronto,43.685347,-79.338106,0.0,Convenience Store,Park,Pizza Place
50,North York,Humber Summit,43.756303,-79.565963,0.0,Pizza Place,Empanada Restaurant,Yoga Studio
63,York,"The Junction North,Runnymede",43.673185,-79.487262,0.0,Pizza Place,Grocery Store,Caribbean Restaurant
70,Etobicoke,Westmount,43.696319,-79.532242,0.0,Pizza Place,Intersection,Sandwich Place
72,North York,Willowdale West,43.782736,-79.442259,0.0,Pharmacy,Grocery Store,Pizza Place
77,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724,0.0,Park,Pizza Place,Mobile Phone Shop


In [292]:
cluester1_target = {'Borough': [],'Neighborhood': [],'Latitude': [],'Longitude': [],'Cluster Labels': [],'1st Most Common Venue': [],'2nd Most Common Venue': [],'3nd Most Common Venue': []}

for i in range(0,len(cluester1)):
    
        
    cluester1_target['Borough'].append(cluester1.iloc[i,0])
    cluester1_target['Neighborhood'].append(cluester1.iloc[i,1])
    cluester1_target['Latitude'].append(cluester1.iloc[i,2])
    cluester1_target['Longitude'].append(cluester1.iloc[i,3])
    cluester1_target['Cluster Labels'].append(cluester1.iloc[i,4])
        
    
    if cluester1.iloc[i,5].find('Restaurant') != -1 or cluester1.iloc[i,5].find('Pizza') != -1 or cluester1.iloc[i,5].find('Food') != -1 or cluester1.iloc[i,5].find('Sandwich') != -1 or cluester1.iloc[i,5].find('Bar') != -1:
        cluester1_target['1st Most Common Venue'].append(0)
        
    else:
        cluester1_target['1st Most Common Venue'].append(1)
            
            
    if cluester1.iloc[i,6].find('Restaurant') != -1 or cluester1.iloc[i,6].find('Pizza') != -1 or cluester1.iloc[i,6].find('Food') != -1 or cluester1.iloc[i,6].find('Sandwich') != -1 or cluester1.iloc[i,6].find('Bar') != -1:
        cluester1_target['2nd Most Common Venue'].append(0)
    else:
        cluester1_target['2nd Most Common Venue'].append(1)
            
            
    if cluester1.iloc[i,7].find('Restaurant') != -1 or cluester1.iloc[i,7].find('Pizza') != -1 or cluester1.iloc[i,7].find('Food') != -1 or cluester1.iloc[i,7].find('Sandwich') != -1 or cluester1.iloc[i,7].find('Bar') != -1:
        cluester1_target['3nd Most Common Venue'].append(0)
    else:
        cluester1_target['3nd Most Common Venue'].append(1)
            
cluester1_target= pd.DataFrame(cluester1_target)
cluester1_target['Total']=cluester1_target.iloc[:,5] + cluester1_target.iloc[:,6] + cluester1_target.iloc[:,7]
cluester1_target=cluester1_target.sort_values(['Total'],ascending=False)
cluester1_target=cluester1_target[cluester1_target['Total']>2]
cluester_final_1=cluester1_target
cluester_final_1.head(100)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3nd Most Common Venue,Total


* We don't have values that satisfy the requirments.  

### Cluster 2:

In [148]:
cluester2=toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluester2.head(10)

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1.0,Fast Food Restaurant,Food & Drink Shop,Park,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
21,York,1.0,Park,Fast Food Restaurant,Women's Store,Pharmacy,Market,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant
40,North York,1.0,Airport,Park,Yoga Studio,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
45,North York,1.0,Park,Yoga Studio,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
66,North York,1.0,Park,Bank,Convenience Store,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
83,Central Toronto,1.0,Park,Restaurant,Playground,Trail,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
85,Scarborough,1.0,Park,Playground,Yoga Studio,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
91,Downtown Toronto,1.0,Park,Playground,Trail,Building,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant
98,Etobicoke,1.0,Park,River,Pool,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run


In [293]:
cluester1=toronto_merged_2.iloc[:,1:9]
cluester1.head(12)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,North York,Parkwoods,43.753259,-79.329656,1.0,Fast Food Restaurant,Food & Drink Shop,Park
21,York,Caledonia-Fairbanks,43.689026,-79.453512,1.0,Park,Fast Food Restaurant,Women's Store
40,North York,"CFB Toronto,Downsview East",43.737473,-79.464763,1.0,Airport,Park,Yoga Studio
45,North York,"Silver Hills,York Mills",43.757490,-79.374714,1.0,Park,Yoga Studio,Dumpling Restaurant
66,North York,York Mills West,43.752758,-79.400049,1.0,Park,Bank,Convenience Store
83,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160,1.0,Park,Restaurant,Playground
85,Scarborough,"Agincourt North,L'Amoreaux East,Milliken,Steel...",43.815252,-79.284577,1.0,Park,Playground,Yoga Studio
91,Downtown Toronto,Rosedale,43.679563,-79.377529,1.0,Park,Playground,Trail
98,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",43.653654,-79.506944,1.0,Park,River,Pool


In [294]:
cluester1_target = {'Borough': [],'Neighborhood': [],'Latitude': [],'Longitude': [],'Cluster Labels': [],'1st Most Common Venue': [],'2nd Most Common Venue': [],'3nd Most Common Venue': []}

for i in range(0,len(cluester1)):
    
        
    cluester1_target['Borough'].append(cluester1.iloc[i,0])
    cluester1_target['Neighborhood'].append(cluester1.iloc[i,1])
    cluester1_target['Latitude'].append(cluester1.iloc[i,2])
    cluester1_target['Longitude'].append(cluester1.iloc[i,3])
    cluester1_target['Cluster Labels'].append(cluester1.iloc[i,4])
        
    
    if cluester1.iloc[i,5].find('Restaurant') != -1 or cluester1.iloc[i,5].find('Pizza') != -1 or cluester1.iloc[i,5].find('Food') != -1 or cluester1.iloc[i,5].find('Sandwich') != -1 or cluester1.iloc[i,5].find('Bar') != -1:
        cluester1_target['1st Most Common Venue'].append(0)
        
    else:
        cluester1_target['1st Most Common Venue'].append(1)
            
            
    if cluester1.iloc[i,6].find('Restaurant') != -1 or cluester1.iloc[i,6].find('Pizza') != -1 or cluester1.iloc[i,6].find('Food') != -1 or cluester1.iloc[i,6].find('Sandwich') != -1 or cluester1.iloc[i,6].find('Bar') != -1:
        cluester1_target['2nd Most Common Venue'].append(0)
    else:
        cluester1_target['2nd Most Common Venue'].append(1)
            
            
    if cluester1.iloc[i,7].find('Restaurant') != -1 or cluester1.iloc[i,7].find('Pizza') != -1 or cluester1.iloc[i,7].find('Food') != -1 or cluester1.iloc[i,7].find('Sandwich') != -1 or cluester1.iloc[i,7].find('Bar') != -1:
        cluester1_target['3nd Most Common Venue'].append(0)
    else:
        cluester1_target['3nd Most Common Venue'].append(1)
            
cluester1_target= pd.DataFrame(cluester1_target)
cluester1_target['Total']=cluester1_target.iloc[:,5] + cluester1_target.iloc[:,6] + cluester1_target.iloc[:,7]
cluester1_target=cluester1_target.sort_values(['Total'],ascending=False)
cluester1_target=cluester1_target[cluester1_target['Total']>2]
cluester_final_2=cluester1_target
cluester_final_2.head(10)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3nd Most Common Venue,Total
2,North York,"CFB Toronto,Downsview East",43.737473,-79.464763,1.0,1,1,1,3
4,North York,York Mills West,43.752758,-79.400049,1.0,1,1,1,3
6,Scarborough,"Agincourt North,L'Amoreaux East,Milliken,Steel...",43.815252,-79.284577,1.0,1,1,1,3
7,Downtown Toronto,Rosedale,43.679563,-79.377529,1.0,1,1,1,3
8,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",43.653654,-79.506944,1.0,1,1,1,3


In [313]:
cluester_final_2.shape

(5, 9)

* We have serveral values that satisfy the requirments.  

### Cluster 3:

In [146]:
cluester3=toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluester3.head(10)

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,2.0,Coffee Shop,Park,Bakery,Café,Pub,Gym / Fitness Center,Breakfast Spot,Mexican Restaurant,Beer Store,Bank
3,North York,2.0,Furniture / Home Store,Clothing Store,Accessories Store,Boutique,Fraternity House,Event Space,Miscellaneous Shop,Coffee Shop,Women's Store,Vietnamese Restaurant
4,Queen's Park,2.0,Coffee Shop,Gym,Park,Hobby Shop,Chinese Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burger Joint,Burrito Place
6,Scarborough,2.0,Fast Food Restaurant,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Field
7,North York,2.0,Gym / Fitness Center,Caribbean Restaurant,Baseball Field,Japanese Restaurant,Café,Convenience Store,Dive Bar,Falafel Restaurant,Event Space,Ethiopian Restaurant
8,East York,2.0,Fast Food Restaurant,Pizza Place,Intersection,Gym / Fitness Center,Pharmacy,Café,Athletics & Sports,Bank,Gastropub,Doner Restaurant
9,Downtown Toronto,2.0,Coffee Shop,Clothing Store,Cosmetics Shop,Fast Food Restaurant,Middle Eastern Restaurant,Café,Ramen Restaurant,Theater,Plaza,Ice Cream Shop
11,Etobicoke,2.0,Bank,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Diner
12,Scarborough,2.0,Bar,Golf Course,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
13,North York,2.0,Grocery Store,Gym,Beer Store,Coffee Shop,Chinese Restaurant,Fast Food Restaurant,Bike Shop,Sporting Goods Shop,Sandwich Place,Japanese Restaurant


In [295]:
cluester1=toronto_merged_3.iloc[:,1:9]
cluester1.head(12)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
2,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,2.0,Coffee Shop,Park,Bakery
3,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,2.0,Furniture / Home Store,Clothing Store,Accessories Store
4,Queen's Park,Queen's Park,43.662301,-79.389494,2.0,Coffee Shop,Gym,Park
6,Scarborough,"Rouge,Malvern",43.806686,-79.194353,2.0,Fast Food Restaurant,Dumpling Restaurant,Discount Store
7,North York,Don Mills North,43.745906,-79.352188,2.0,Gym / Fitness Center,Caribbean Restaurant,Baseball Field
8,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937,2.0,Fast Food Restaurant,Pizza Place,Intersection
9,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,2.0,Coffee Shop,Clothing Store,Cosmetics Shop
11,Etobicoke,"Cloverdale,Islington,Martin Grove,Princess Gar...",43.650943,-79.554724,2.0,Bank,Yoga Studio,Dumpling Restaurant
12,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,2.0,Bar,Golf Course,Yoga Studio
13,North York,"Flemingdon Park,Don Mills South",43.725900,-79.340923,2.0,Grocery Store,Gym,Beer Store


In [297]:
cluester1_target = {'Borough': [],'Neighborhood': [],'Latitude': [],'Longitude': [],'Cluster Labels': [],'1st Most Common Venue': [],'2nd Most Common Venue': [],'3nd Most Common Venue': []}

for i in range(0,len(cluester1)):
    
        
    cluester1_target['Borough'].append(cluester1.iloc[i,0])
    cluester1_target['Neighborhood'].append(cluester1.iloc[i,1])
    cluester1_target['Latitude'].append(cluester1.iloc[i,2])
    cluester1_target['Longitude'].append(cluester1.iloc[i,3])
    cluester1_target['Cluster Labels'].append(cluester1.iloc[i,4])
        
    
    if cluester1.iloc[i,5].find('Restaurant') != -1 or cluester1.iloc[i,5].find('Pizza') != -1 or cluester1.iloc[i,5].find('Food') != -1 or cluester1.iloc[i,5].find('Sandwich') != -1 or cluester1.iloc[i,5].find('Bar') != -1:
        cluester1_target['1st Most Common Venue'].append(0)
        
    else:
        cluester1_target['1st Most Common Venue'].append(1)
            
            
    if cluester1.iloc[i,6].find('Restaurant') != -1 or cluester1.iloc[i,6].find('Pizza') != -1 or cluester1.iloc[i,6].find('Food') != -1 or cluester1.iloc[i,6].find('Sandwich') != -1 or cluester1.iloc[i,6].find('Bar') != -1:
        cluester1_target['2nd Most Common Venue'].append(0)
    else:
        cluester1_target['2nd Most Common Venue'].append(1)
            
            
    if cluester1.iloc[i,7].find('Restaurant') != -1 or cluester1.iloc[i,7].find('Pizza') != -1 or cluester1.iloc[i,7].find('Food') != -1 or cluester1.iloc[i,7].find('Sandwich') != -1 or cluester1.iloc[i,7].find('Bar') != -1:
        cluester1_target['3nd Most Common Venue'].append(0)
    else:
        cluester1_target['3nd Most Common Venue'].append(1)
            
cluester1_target= pd.DataFrame(cluester1_target)
cluester1_target['Total']=cluester1_target.iloc[:,5] + cluester1_target.iloc[:,6] + cluester1_target.iloc[:,7]
cluester1_target=cluester1_target.sort_values(['Total'],ascending=False)
cluester1_target=cluester1_target[cluester1_target['Total']>2]
cluester_final_3=cluester1_target
cluester_final_3.head(100)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3nd Most Common Venue,Total
0,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,2.0,1,1,1,3
51,Central Toronto,Davisville North,43.712751,-79.390197,2.0,1,1,1,3
33,Downtown Toronto,"Design Exchange,Toronto Dominion Centre",43.647177,-79.381576,2.0,1,1,1,3
34,West Toronto,"Brockton,Exhibition Place,Parkdale Village",43.636847,-79.428191,2.0,1,1,1,3
36,North York,Downsview West,43.739015,-79.506944,2.0,1,1,1,3
1,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,2.0,1,1,1,3
39,North York,"Downsview,North Park,Upwood Park",43.713756,-79.490074,2.0,1,1,1,3
42,East Toronto,Studio District,43.659526,-79.340923,2.0,1,1,1,3
45,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848,2.0,1,1,1,3
47,North York,Downsview Northwest,43.761631,-79.520999,2.0,1,1,1,3


In [312]:
cluester_final_3.shape

(37, 9)

* We have a lot of values that satisfy the requirments.  

### Cluster 4:

In [151]:
cluester4=toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluester4.head(10)

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,3.0,Baseball Field,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Diner


In [298]:
cluester1=toronto_merged_4.iloc[:,1:9]
cluester1.head(12)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
57,North York,"Emery,Humberlea",43.724766,-79.532242,3.0,Baseball Field,Yoga Studio,Dumpling Restaurant


In [299]:
cluester1_target = {'Borough': [],'Neighborhood': [],'Latitude': [],'Longitude': [],'Cluster Labels': [],'1st Most Common Venue': [],'2nd Most Common Venue': [],'3nd Most Common Venue': []}

for i in range(0,len(cluester1)):
    
        
    cluester1_target['Borough'].append(cluester1.iloc[i,0])
    cluester1_target['Neighborhood'].append(cluester1.iloc[i,1])
    cluester1_target['Latitude'].append(cluester1.iloc[i,2])
    cluester1_target['Longitude'].append(cluester1.iloc[i,3])
    cluester1_target['Cluster Labels'].append(cluester1.iloc[i,4])
        
    
    if cluester1.iloc[i,5].find('Restaurant') != -1 or cluester1.iloc[i,5].find('Pizza') != -1 or cluester1.iloc[i,5].find('Food') != -1 or cluester1.iloc[i,5].find('Sandwich') != -1 or cluester1.iloc[i,5].find('Bar') != -1:
        cluester1_target['1st Most Common Venue'].append(0)
        
    else:
        cluester1_target['1st Most Common Venue'].append(1)
            
            
    if cluester1.iloc[i,6].find('Restaurant') != -1 or cluester1.iloc[i,6].find('Pizza') != -1 or cluester1.iloc[i,6].find('Food') != -1 or cluester1.iloc[i,6].find('Sandwich') != -1 or cluester1.iloc[i,6].find('Bar') != -1:
        cluester1_target['2nd Most Common Venue'].append(0)
    else:
        cluester1_target['2nd Most Common Venue'].append(1)
            
            
    if cluester1.iloc[i,7].find('Restaurant') != -1 or cluester1.iloc[i,7].find('Pizza') != -1 or cluester1.iloc[i,7].find('Food') != -1 or cluester1.iloc[i,7].find('Sandwich') != -1 or cluester1.iloc[i,7].find('Bar') != -1:
        cluester1_target['3nd Most Common Venue'].append(0)
    else:
        cluester1_target['3nd Most Common Venue'].append(1)
            
cluester1_target= pd.DataFrame(cluester1_target)
cluester1_target['Total']=cluester1_target.iloc[:,5] + cluester1_target.iloc[:,6] + cluester1_target.iloc[:,7]
cluester1_target=cluester1_target.sort_values(['Total'],ascending=False)
cluester1_target=cluester1_target[cluester1_target['Total']>2]
cluester_final_4=cluester1_target
cluester_final_4.head(100)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3nd Most Common Venue,Total


* We don't have values that satisfy the requirments.  

# Neighborhood that satisfy the requirements

In [314]:
cluester_final=pd.concat([cluester_final_1,cluester_final_2,cluester_final_3,cluester_final_4])
cluester_final.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3nd Most Common Venue,Total
2,North York,"CFB Toronto,Downsview East",43.737473,-79.464763,1.0,1,1,1,3
4,North York,York Mills West,43.752758,-79.400049,1.0,1,1,1,3
6,Scarborough,"Agincourt North,L'Amoreaux East,Milliken,Steel...",43.815252,-79.284577,1.0,1,1,1,3
7,Downtown Toronto,Rosedale,43.679563,-79.377529,1.0,1,1,1,3
8,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",43.653654,-79.506944,1.0,1,1,1,3


# Create a map to see the best places to open the restaurant

In [339]:
# create map
latitude= 43.647177881471634
longitude= -79.3822181224823
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=14)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cluester_final['Latitude'], cluester_final['Longitude'], cluester_final['Neighborhood'], cluester_final['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7).add_to(map_clusters)
    
folium.Circle([latitude, longitude], 400, fill=True,fill_color='green',color='green').add_child(folium.Popup('My name is Circle')).add_to(map_clusters)  
            
map_clusters

# Conclusion:
## After the deeply analysis we highly recommended to open the new restaurant in the Downtown, especially we see enormous opportunities in the financial district. 